In [1]:
import sys
sys.path.append("../")
from pyproj import CRS
import numpy as np
import sys
from osgeo import gdal
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import colors
import matplotlib.dates as mdates
import matplotlib as mpl
from matplotlib.ticker import FuncFormatter
from datetime import datetime
from time import time
from collections import Counter
import pandas as pd #for data analysing
import openpyxl
import os



In [3]:
###j'ouvre mon dataset
NDSI_SON = xr.open_dataset('NDSI_DJF_monthlyproba_b_nc.nc')


<xarray.Dataset>
Dimensions:      (longitude: 7117, latitude: 6200, z: 105)
Coordinates:
  * longitude    (longitude) float64 8.125 8.126 8.126 ... 10.55 10.55 10.55
  * latitude     (latitude) float64 47.85 47.85 47.85 ... 45.74 45.74 45.74
  * z            (z) datetime64[ns] 1984-12-01 1985-01-01 ... 2021-12-01
Data variables:
    DEM          (latitude, longitude) float32 ...
    snow_final3  (z, latitude, longitude) float32 ...
    snow_final4  (z, latitude, longitude) float32 ...
    crs          int32 ...
Attributes:
    Conventions:  CF-1.4
    created_by:   R, packages ncdf4 and raster (version 3.6-23)
    date:         2024-02-08 11:17:36
    crs:          +proj=longlat +datum=WGS84 +no_defs +type=crs

In [4]:
####j'attribue un CRS à mon dataset
crs = CRS.from_string('EPSG:4326')
NDSI_SON.attrs["crs"] = crs.to_proj4()

/home/charlottepoussin/.conda/envs/myenv/lib/python3.12/site-packages/pyproj/crs/crs.py:1293: UserWarning: You will likely lose important projection information when converting to a PROJ string from another format. See: https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
  proj = self._crs.to_proj4(version=version)


#### On va donner comme noms les dates car pour le moment c'est 1,2,3,4

In [6]:
###j'ouvre le DEM
DEM100_a = xr.open_dataset('DEM100_b.nc')

In [7]:
DEM100_a = DEM100_a.isel(z=0)

In [9]:
DEM100_a.attrs["crs"] = crs.to_proj4()

In [10]:
###on veut metre le DEM dans le dataset
lat = NDSI_SON['latitude'].values
lon = NDSI_SON['longitude'].values
time = NDSI_SON['z'].values

dem = xr.DataArray(DEM100_a.layer.values, coords={'latitude': lat,'longitude':lon}, dims=['latitude','longitude'])

In [11]:
NDSI_SON['DEM100'] = dem

In [12]:
####on change en entier le DEM on a pas besoisn de virgule
NDSI_SON['DEM100'] = NDSI_SON.DEM100.astype('int16')

/home/charlottepoussin/.conda/envs/myenv/lib/python3.12/site-packages/xarray/core/duck_array_ops.py:213: RuntimeWarning: invalid value encountered in cast
  return data.astype(dtype, **kwargs)


In [14]:
##les valeur hors suisse son en 0 et non en NAN por le DEM on va alors les changer en NAN
NDSI_SON['DEM100'] = NDSI_SON.DEM100.where(NDSI_SON.DEM100 != 0)

### Snowline step3
On commence le process

#### On défini les fonctions

In [17]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##Before
def compute_cloudpercent_b(array):
    cloud = np.sum(array == 2)
    #print(cloud)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    percentage_cloud_b = round(((float(cloud) / float(tot_pix)) * 100),2)
    return percentage_cloud_b

In [18]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##Before
def compute_snowpercent_b(array):
    snow = np.count_nonzero((array == 1) | (array == 100))
    #print(snow)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    snow_percent_b = round(((float(snow) / float(tot_pix)) * 100),2)
    return snow_percent_b

In [19]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##Before
def compute_landpercent_b(array):
    land = np.count_nonzero((array == 0) | (array == 200))
    #print(land)
    tot_pix = np.count_nonzero((~np.isnan(array)))
    land_percent_b = round((float(land) / float(tot_pix)) * 100,2)
    return land_percent_b

In [20]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_cloudpercent_a(array_copy):
    cloud = np.sum(array_copy  == 2)
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    cloud_percent = round((float(cloud) / float(tot_pix)) * 100, 2)
    return cloud_percent

In [21]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_snowpercent_a(array_copy):
    snow = np.count_nonzero((array_copy == 1) | (array_copy == 100))
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    snow_percent = round((float(snow) / float(tot_pix)) * 100, 2)
    return snow_percent

In [22]:
##le faire avant l'analyse et après l'analyse afin de voir le pourcentage de changement
##After
def compute_landpercent_a(array_copy):
    land = np.count_nonzero((array_copy == 0) | (array_copy == 200))
    tot_pix = np.count_nonzero((~np.isnan(array_copy)))
    land_percent = round((float(land) / float(tot_pix)) * 100, 2)
    return land_percent

In [23]:
def step3_gapfilling_cloud(array,array_copy):
    cloud_b = np.sum(array==2)
    cloud_a = np.sum(array_copy==2)
    cloud_step3 = cloud_b - cloud_a 
    step3_gf_cloud = round((float(cloud_step3)/float(cloud_b))*100,2)
    #step3_gf_cloud = ((cloud_step3)/(cloud_b))*100
    return step3_gf_cloud

In [24]:
def step3_gapfilling_tot(array,array_copy):
    cloudpercent_b = compute_cloudpercent_b(array)
    cloudpercent_a = compute_cloudpercent_a(array_copy)
    step3_gf_tot = round(((cloudpercent_b)-(cloudpercent_a)),2)
    #step3_gf_tot = ((cloudpercent_b)-(cloudpercent_a))
    return step3_gf_tot

In [25]:
##on va prendre une bande d'altitude en dessous pour avoir une marge d'erreur
###ici on a une condition en plus disant que l'on veut avoir pour le HSmin au koins 10% de donnée clear
### et on rajoute qu'il doit y avoir au moins 10% de neige pour ne pas prendre l'altitude ou le neige est confondu avec l'eau
def compute_HSmin(array, dem_array):
    percentage_snow = ((array == 1) | (array == 100)).groupby(dem_array).mean() * 100
    percentage_land = ((array == 0) | (array == 200)).groupby(dem_array).mean() * 100
    percentage_clear = percentage_snow + percentage_land
    valid_altitudes = percentage_clear.where(percentage_clear > 10).dropna(dim='DEM100').DEM100
    valid_elements = xr.where(dem_array.isin(valid_altitudes).astype(int), 1, 0)
    update_indices = percentage_snow.where(percentage_snow > 0.1).dropna(dim='DEM100').DEM100
    valid_indices = xr.where(dem_array.isin(update_indices).astype(int), 1, 0)
    cond = (dem_array.values[((array.values == 1) | (array.values == 100)) & (valid_elements.values ==1) & (valid_indices.values ==1)])
   
    if len(cond) > 0:
        HS_min = (cond.min())
    else: 
        # Aucune altitude valide trouvée, vous pouvez gérer cela comme vous le souhaitez
        #print("Aucune altitude valide trouvée pour condmin.")
        HS_min = None  # Ou attribuez une valeur spéciale ou laissez HS_max à None
    
    return HS_min

In [26]:
compute_HSmin(NDSI_SON.snow_final5[1], NDSI_SON.DEM100)

300.0

In [27]:
##on va prendre une bande d'altitude en dessus pour avoir une marge d'erreur
###ici on a une condition en plus disant que l'on veut avoir pour le HSmin au koins 10% de donnée clear
def compute_HSmax(array, dem_array):
    percentage_snow = ((array == 1) | (array == 100)).groupby(dem_array).mean() * 100
    percentage_land = ((array == 0) | (array == 200)).groupby(dem_array).mean() * 100
    percentage_clear = percentage_snow + percentage_land
    valid_altitudes = percentage_clear.where(percentage_clear > 10).dropna(dim='DEM100').DEM100
    valid_elements = xr.where(dem_array.isin(valid_altitudes).astype(int), 1, 0)
    elev_max_nosnow = np.nanmax(dem_array.values[((array.values == 0) | (array.values == 200))])
    condmax = (dem_array.values[((array.values == 1) | (array.values == 100)) & (
    dem_array.values >= elev_max_nosnow) & (valid_elements.values ==1)])

    if len(condmax) > 0:
        HS_max = (condmax.min())
    else:
    # Aucune altitude valide trouvée, vous pouvez gérer cela comme vous le souhaitez
        #print("Aucune altitude valide trouvée pour condmax.")
        HS_max = None  # Ou attribuez une valeur spéciale ou laissez HS_max à None
    
    return HS_max

In [28]:
def step3_Option1(array, dem_array):

    percentage_cloud_b = compute_cloudpercent_b(array)
    snow_percent_b = compute_snowpercent_b(array)
    #print("cloud percent before step 3 : " + str(percentage_cloud_b))
    print("cloud percent before step 3 : " + str(round(percentage_cloud_b,2)))
   
    # Perform step 2 only if cloud coverage is less than a threshold value (40)
    # Perform step 2 only if snowverage is larger a threshold value (1)
    # (hard coded for now to 30%)
    cloudpercent_condition = percentage_cloud_b <= 40
    snowpercent_condition = snow_percent_b >= 1
    

    if cloudpercent_condition and snowpercent_condition:
        #je crée une copie pour pouvoir travailler dessus sans changer l'initial
        snow_step3 = array.copy(deep=True)
        
        hs_min = compute_HSmin(snow_step3, dem_array)
        print("HS_min values for each z: " + str(hs_min))
        # S(y,x,t) = 0 if (H(x,y) < Hsmin(t))

        if hs_min is not None:
            condition_min = ((snow_step3.values==2) &
             (dem_array.values < hs_min))
            snow_step3.values[condition_min]=200
           #print(condition_min)
        else:
            print("No valid altitude found for HS_min.")
        
        hs_max = compute_HSmax(snow_step3, dem_array)
        print("HS_max values for each z: " + str(hs_max))

        if hs_max is not None:
        # S(y,x,t) = 100 if (H(x,y) > Hsmax(t))
            condition_max = ((snow_step3.values==2) &
             (dem_array.values > hs_max))
            snow_step3.values[condition_max]=100
        else:
            print("No valid altitude found for HS_max.")
    
        percentage_cloud_a = compute_cloudpercent_a(snow_step3)
        print("cloud percent after step 3 : " + str(round(percentage_cloud_a,2)))
    
        step3_gf_tot = step3_gapfilling_tot(array,snow_step3)
        #print("diminution of cloud tot (%) : " + str((step3_gf_tot)))
        print("diminution of cloud tot (%) : " + str(round(step3_gf_tot,2)))
        step3_gf_cloud = step3_gapfilling_cloud(array,snow_step3)
        #print("diminution of cloud (%) : " + str((step3_gf_cloud)))
        print("diminution of cloud (%) : " + str(round(step3_gf_cloud,2)))

        #datasetarray['snow_step5'] = snow_step5
        #snow_step6 = snow_step5
    else:
        return cloudpercent_condition, array
    
    return cloudpercent_condition, snow_step3

In [29]:
hs_min_list = []
hs_max_list = []
cloud_percent_b_list = []
snow_percent_b_list = []
land_percent_b_list = []
cloud_percent_a_list = []
snow_percent_a_list = []
land_percent_a_list = []
step3_gapfilling_tot_list = []
step3_gapfilling_cloud_list = []
time_list = []
cloud_conditions = []
snow_step3_list = []

In [31]:
snow_step3_concat = xr.concat(snow_step3_list, dim='z')
dataset_step3 = xr.Dataset({'snow_step3': snow_step3_concat})

In [34]:
data = {
    'Time': time_list,
    'Cloud Condition': cloud_conditions,
    'Cloud Percent Before': cloud_percent_b_list,
    'Snow Percent Before': snow_percent_b_list,
    'Land Percent Before': land_percent_b_list,
    'Cloud Percent After': cloud_percent_a_list,
    'Snow Percent After': snow_percent_a_list,
    'Land Percent After': land_percent_a_list,
    'HS Min': hs_min_list,
    'HS Max': hs_max_list,
    'Cloud Reduction (%)': step3_gapfilling_tot_list,
    'Cloud Reduction After (%)': step3_gapfilling_cloud_list,
}

# Créer un DataFrame à partir du dictionnaire
df_step3 = pd.DataFrame(data)
filename = f'df_snowline_DJF_b.xlsx'
filepath = os.path.join('/home/charlottepoussin/Excels', filename)
df_step3.to_excel(filepath, index=False)

###j'ouvre mon étude de cas
CS = xr.open_dataset('NDSI_SON_beforSL_b_nc.nc')